In [2]:
!pip install google-generativeai


In [3]:
!pip install pymupdf

In [2]:
import google.generativeai as genai

In [3]:
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


In [4]:
api_key = os.getenv("GOOGLE_API_KEY")


In [5]:
genai.configure(api_key=api_key)

# Initialize the Gemini model
model = genai.GenerativeModel("gemini-2.0-flash")

# Ask a question
response = model.generate_content("Explain quantum computing in simple terms.")

# Print the response
print(response.text)

Imagine a light switch. A regular computer bit is like that light switch: it can be either ON (1) or OFF (0). It's a definitive state.

Quantum computing is different. Instead of a simple light switch, imagine a dimmer switch that can be anywhere between fully ON and fully OFF, and even both at the same time! This "both at the same time" state is called **superposition**.  Think of it like a coin spinning in the air - it's neither heads nor tails until it lands.

Here's the breakdown:

* **Qubit (Quantum Bit):**  Instead of a regular bit (0 or 1), a quantum bit (qubit) can be 0, 1, or a **superposition** of both. This "in-between" state allows quantum computers to explore many possibilities simultaneously.

* **Superposition:**  The ability of a qubit to exist in multiple states at the same time. This dramatically increases the potential computational power because it allows the computer to explore many possibilities simultaneously instead of one at a time.

* **Entanglement:** This is

In [102]:
import fitz
import difflib
import re
import json

In [7]:
def read_pdf(file_path):
    """Reads a PDF file"""
    doc = fitz.open(file_path)
    return "\n".join([page.get_text() for page in doc])

def read_legal_files(folder_path):
    """Reads all legal files in the folder and returns a dictionary with file names and content."""
    legal_documents = {}

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith(".pdf"):
            legal_documents[file_name] = read_pdf(file_path)
        else:
            print(f"Skipping unsupported file: {file_name}")

    return legal_documents

In [18]:
folder_path = "full_contract_pdf/Part_I/Affiliate_Agreements/"
legal_docs = read_legal_files(folder_path)

# Display first document
for file_name, content in legal_docs.items():
    print(f"--- {file_name} ---\n{content[:500]}...\n")
    break  # Only show the first one for preview

Skipping unsupported file: perturbed_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf.txt
Skipping unsupported file: .ipynb_checkpoints
--- UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf ---
                                                                   EXHIBIT 10.11
                         NETWORK 1 FINANCIAL CORPORATION
                           AFFILIATE OFFICE AGREEMENT
THIS  AGREEMENT  is  entered  into  by  and  between  NETWORK  1 FINANCIAL, INC.
("NETWORK  1"),  a  Virginia Corporation with its principal place of business at
1501  Farm  Credit  Drive,  Suite 1500, McLean, Virginia 22102-5004, and Payment
Data  Systems,  Inc.,  the  Affiliate Office ("AFFILIATE"), a Nev...



In [19]:
for file_name, content in legal_docs.items():
    print(f"--- {file_name} ---...\n")


--- UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf ---...

--- DigitalCinemaDestinationsCorp_20111220_S-1_EX-10.10_7346719_EX-10.10_Affiliate Agreement.pdf ---...

--- LinkPlusCorp_20050802_8-K_EX-10_3240252_EX-10_Affiliate Agreement.pdf ---...

--- CreditcardscomInc_20070810_S-1_EX-10.33_362297_EX-10.33_Affiliate Agreement.pdf ---...

--- SouthernStarEnergyInc_20051202_SB-2A_EX-9_801890_EX-9_Affiliate Agreement.pdf ---...

--- UnionDentalHoldingsInc_20050204_8-KA_EX-10_3345577_EX-10_Affiliate Agreement.pdf ---...

--- SteelVaultCorp_20081224_10-K_EX-10.16_3074935_EX-10.16_Affiliate Agreement.pdf ---...

--- TubeMediaCorp_20060310_8-K_EX-10.1_513921_EX-10.1_Affiliate Agreement.pdf ---...

--- CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf ---...



In [28]:
def normalize_text(text):
    """
    Normalizes text by removing extra spaces, line breaks, and ensuring consistent spacing.
    """
    text = text.replace("\n", " ")  # Replace newlines with space
    text = re.sub(r"\s+", " ", text).strip()  # Replace multiple spaces with a single space
    return text

In [125]:
def apply_perturbation_from_json(original_text, json_file, output_folder="perturbed_legal_documents/inconsistencies_inText_contradiction/modified_files_tags/"):
    """
    Reads the JSON metadata and applies the described perturbations to the original document,
    adding unique <*$p$*> markers around the modified sections.

    Parameters:
    - original_text (str): The original contract text.
    - json_file (str): Path to the JSON file containing the perturbation details.
    - output_folder (str): Folder to save the modified contract.

    Returns:
    - modified_text (str): The full modified document.
    """

    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)
    
    print("This is json file:", json_file)
    
    # Load the JSON metadata
    with open(json_file, "r", encoding="utf-8") as file:
        json_data = json.load(file)

    print("file loaded") 
    if isinstance(json_data, list) and len(json_data) > 0:
        json_data = json_data[0]  # Extract the first item in the list
    
    # Normalize the original contract text
    normalized_text = normalize_text(original_text)

    # Apply modifications with unique markers
    modified_text = normalized_text
    
    for perturbation in json_data["perturbation"]:
        original_section = normalize_text(perturbation["original_text"])  
        #print("this is original text:", original_section)
        changed_section = normalize_text(perturbation["changed_text"])
        #print("this is the changed text:", changed_section)
        
        # Wrap changed section with unique <*$p$*> markers
        marked_section = f"<*$p$*>{changed_section}<*$p$*>"

        # Replace original section with marked modified section
        if original_section in modified_text:
            modified_text = modified_text.replace(original_section, marked_section)
        else:
            print(f"Warning: Could not find section in text: {original_section}")
            error = "COULD NOT MODIFY FILE"
            return error

    # Save the modified contract as a new file
    modified_file_name = f"modified_{json_data['file_name']}.txt"
    modified_file_path = os.path.join(output_folder, modified_file_name)

    with open(modified_file_path, "w", encoding="utf-8") as file:
        file.write(modified_text)

    return modified_text

In [124]:
def generate_perturbation_new(original_text, file_name):    
    """Generates a perturbed version of the legal document section in structured JSON format."""
    
    prompt_inconsistencies_inText = f"""You are a financial contract auditor ensuring consistency in payment terms. Your task is to introduce a conflict between different payment deadline clauses.

    ### **Definition:**
    Inconsistencies also lead to **in-text contradictions** when **different sections of a contract provide conflicting deadlines, obligations, or penalties**. This creates ambiguity regarding which terms should be enforced, leading to disputes over contractual obligations.

    ### **Step-by-Step Instructions:**
    1. Identify **two different sections** where payment terms or penalties are mentioned.  
    2. Modify one section so that it **conflicts with the other**.  
    3. Ensure the change **creates uncertainty in enforcement**.  
    4. Output the modified contract in structured JSON format.  

    ---

    ### **Examples of Conflicting Payment Terms:**

    **Example 1:**
    - **Section 4:** "Invoices must be paid within 45 days."
    - **Section 12:** "Late fees apply if invoices are unpaid after 30 days."
    - **Explanation:** One section allows **45 days**, but the other **charges late fees after 30 days**, creating ambiguity.

    **Example 2:**
    - **Section 6:** "Security deposits are fully refundable."
    - **Section 9:** "Security deposits are non-refundable."
    - **Explanation:** It is unclear whether **deposits must be returned**.

    **Example 3:**
    - **Section 3:** "Early termination fees apply if the contract is ended before 12 months."
    - **Section 10:** "Clients may terminate at any time without penalty."
    - **Explanation:** Conflicting clauses **cause uncertainty about termination fees**.

    **Example 4:**
    - **Section 5:** "Payment plans must be completed within 6 months."
    - **Section 11:** "Customers may extend payment plans up to 12 months."
    - **Explanation:** Different sections **contradict the maximum payment period**.

    **Example 5:**
    - **Section 8:** "Interest on late payments accrues at 5% per month."
    - **Section 14:** "No interest will be charged on late payments."
    - **Explanation:** **Conflicting interest policies** could lead to legal disputes.

    **Example 6:**
    - **Original:** "For the purposes of this Agreement, the term 'Affiliate' shall refer to any entity that is controlled by, controls, or is under common control with either party. The term 'control' shall mean ownership of at least fifty-one percent (51%) of the voting equity in such entity. Affiliates shall be entitled to the same rights and obligations as the contracting party."
    - **Modified:** "For the purposes of this Agreement, the term 'Affiliate' shall refer to any entity that is affiliated with, works in conjunction with, or has a strategic partnership with either party. Affiliates may be granted certain rights and obligations as determined by the contracting party on a case-by-case basis."
    - **Explanation:** The modified text **creates an in-text contradiction** by **changing the definition of "Affiliate"** to include strategic partnerships and loosely affiliated entities, rather than just entities **under majority control**. This contradicts other contract sections that **rely on the original definition** (51% ownership) to determine rights and obligations. Additionally, **allowing case-by-case determination** undermines the original intent of treating affiliates the same as contracting parties, introducing **legal uncertainty** and **potential disputes over obligations**.
    ---

    ### **Return JSON Format**
    {{
        "file_name": {file_name},
        "perturbation": [
            {{
                "type": "Inconsistencies - In Text Contradiction",
                "original_text": "EXCERPT BEFORE CHANGE",
                "changed_text": "EXCERPT AFTER CHANGE",
                "explanation": "WHY THIS CHANGE INTRODUCES A PERTURBATION",
                "location": "SECTION OR PARAGRAPH NUMBER"
            }}
        ]
    }}
    
    Below is the original legal text:
    -------------------
    {original_text}
    -------------------

    Now, return ONLY the structured JSON object with the modified text and explanation.
    """
    response = model.generate_content(prompt_inconsistencies_inText)
    
    if not response:
        return "ERROR: No response from API"

    return response.text if response else "ERROR: No response from API"

In [21]:
def highlight_changes(original, modified):
    """Compares original and modified text and marks changes."""
    original_lines = original.split("\n")
    modified_lines = modified.split("\n")

    diff = difflib.ndiff(original_lines, modified_lines)
    highlighted = []
    
    for line in diff:
        if line.startswith("+ "):  # Added text
            highlighted.append(f"[MODIFIED] {line[2:]}")
        elif line.startswith("- "):  # Removed text
            highlighted.append(f"[REMOVED] {line[2:]}")
        else:
            highlighted.append(line[2:])  
    
    return "\n".join(highlighted)

In [22]:
def extract_clean_text(perturbed_text):
    """
    Removes [MODIFIED], [REMOVED] tags and explanations, leaving only the modified version.
    """
    # Remove any [MODIFIED] or [REMOVED] markers
    clean_text = re.sub(r"\[MODIFIED\]|\[REMOVED\]", "", perturbed_text)
    
    # Remove explanations (assuming they are after a certain marker like "Explanation:")
    clean_text = re.sub(r"Explanation:.*", "", clean_text, flags=re.DOTALL)
    
    # Clean up extra spaces that may remain after removal
    clean_text = re.sub(r"\n\s*\n", "\n", clean_text).strip()
    
    
    
    return clean_text

In [126]:
def apply_perturbations(folder_path, perturbation_type="contradiction"):
    legal_docs = read_legal_files(folder_path)
    

    for i, (file_name, content) in enumerate(legal_docs.items()):
        results = []
        if i >= 10:  # Stop after processing 5 documents
             break
        print(f"Processing {file_name}...")
        #perturbed_json = generate_perturbation(content, file_name, perturbation_type)
        perturbed_json = generate_perturbation_new(content, file_name)
        #print("This is the perturbed json:", perturbed_json)
        clean_json_text = re.sub(r"```json|```", "", perturbed_json).strip()

        # print('this is json:', clean_json_text)
        try:
            # Convert response into a Python dictionary
            perturbed_data = json.loads(clean_json_text)
            results.append(perturbed_data)
        except json.JSONDecodeError:
            print(f"Error parsing JSON for {file_name}, skipping...")
            continue
            
            
        # Save the modified text
        folder_path = "perturbed_legal_documents/inconsistencies_inText_contradiction/"

        # Save the JSON output
        json_output_path = os.path.join(folder_path, f"perturbed_{file_name}.json")
        with open(json_output_path, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        
        json_file_path = f"perturbed_legal_documents/inconsistencies_inText_contradiction/perturbed_{file_name}.json"
        modified_contract = apply_perturbation_from_json(content, json_file_path)
        print("________________________________________________________________________")

    print(f"All perturbations saved in {folder_path}")

    return perturbed_json

In [130]:
folder_path = "full_contract_pdf/Part_I/Affiliate_Agreements/"
perturbation_type = "contradiction"  # Change to "ambiguity", "omission", etc.
perturbed_legal_docs = apply_perturbations(folder_path, perturbation_type)

Skipping unsupported file: perturbed_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf.txt
Skipping unsupported file: .ipynb_checkpoints
Processing UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf...
This is json file: perturbed_legal_documents/inconsistencies_inText_contradiction/perturbed_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf.json
file loaded
________________________________________________________________________
Processing DigitalCinemaDestinationsCorp_20111220_S-1_EX-10.10_7346719_EX-10.10_Affiliate Agreement.pdf...
This is json file: perturbed_legal_documents/inconsistencies_inText_contradiction/perturbed_DigitalCinemaDestinationsCorp_20111220_S-1_EX-10.10_7346719_EX-10.10_Affiliate Agreement.pdf.json
file loaded
________________________________________________________________________
Processing LinkPlusCorp_20050802_8-K_EX-10_3240252_EX-10_Affiliate Agreement.pdf...
This is json file: perturbed_

In [63]:
json_text = """{
    "text": "hello",
    "explain": [
        {
            "from": "tucson"
        }
    ]
}"""


In [61]:
json_output = json.loads(json_text)  # Converts to JSON with formatting
print(json_output)

{'text': 'hello', 'explain': [{'from': 'tucson'}]}


In [41]:
print(results)

[<module 'json' from '/Users/adithmouli/anaconda3/lib/python3.11/json/__init__.py'>]
